In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print 'Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER)

from gensim import corpora

Folder "/var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T" will be used to save temporary dictionary and corpus.


2017-07-21 11:39:32,003 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [10]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

# lemmatization - get original form of words ????

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1] for text in texts]
from pprint import pprint
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [12]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
print dictionary

2017-07-21 14:46:46,748 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-07-21 14:46:46,749 : INFO : built Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...) from 9 documents (total 29 corpus positions)
2017-07-21 14:46:46,751 : INFO : saving Dictionary object under /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.dict, separately None
2017-07-21 14:46:46,752 : INFO : saved /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.dict


Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


In [21]:
print dictionary.token2id

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [27]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)
corpus

2017-07-21 15:44:16,571 : INFO : storing corpus in Matrix Market format to /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm
2017-07-21 15:44:16,572 : INFO : saving sparse matrix to /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm
2017-07-21 15:44:16,573 : INFO : PROGRESS: saving document #0
2017-07-21 15:44:16,575 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-07-21 15:44:16,575 : INFO : saving MmCorpus index to /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]

In [32]:
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))
print list(corpus)

2017-07-21 15:52:44,050 : INFO : loaded corpus index from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm.index
2017-07-21 15:52:44,051 : INFO : initializing corpus reader from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm
2017-07-21 15:52:44,053 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


[[(0, 1.0), (1, 1.0), (2, 1.0)], [(1, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)], [(0, 1.0), (6, 1.0), (7, 1.0), (8, 1.0)], [(2, 1.0), (6, 2.0), (8, 1.0)], [(3, 1.0), (4, 1.0), (7, 1.0)], [(9, 1.0)], [(9, 1.0), (10, 1.0)], [(9, 1.0), (10, 1.0), (11, 1.0)], [(5, 1.0), (10, 1.0), (11, 1.0)]]


In [36]:
from gensim import models
tfidf = models.TfidfModel(corpus)

2017-07-21 16:22:26,956 : INFO : collecting document frequencies
2017-07-21 16:22:26,958 : INFO : PROGRESS: processing document #0
2017-07-21 16:22:26,959 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [47]:
for i in tfidf[corpus]:
    print i

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]
